# Train the Spire

Final Project  <br />
Daniel Feldman

**Project Decription**:

My goal in this project is to apply machine learning to a card game called “Slay the Spire.” This game is a deck building card game where a player must try to survive a series of combats without losing all of their health points. The final result of the project will be to create a model that given a player’s ingame progress, will be able to predict the amount of health lost in subsequent combats. This will allow the player to identify the most effective card to add to their deck at each point in the game.

**Data Processing**:

I transformed the categorical data to quantitative data using both a multi-label binarizer and a count vectorizer. I used a multi-label binarizer rather than a one-hot encoder since I needed to encode multiple labels per instance, for example each player has a set of various different relics. The multi-label binarizer allowed me to convert the relic data into a binary vector. Each position in the vector is associated with a specific relic. The value at a position represents whether the player has that specific relic: 1 means they have the relic , 0 means they do not have the relic.

I repeated a similar process for transforming the player cards and enemies fought, except in this case, a player may have multiple copies of the same card or may have fought the same enemy more than once. Therefore, count vectorizer allows me to determine the total number of occurrences for each card and enemy. Health and floor values were already numerical values, so they did not need to be converted into vectors. To create the final feature vector, I concatenate the cards, enemies, and relics encodings with the floor and health values. The target value in this set is the average damage taken per combat. The final model is a multilayer perceptron that takes the players attributes (their relics, cards, health, enemies fought,etc.) and calculates the average damage they will incur in future fights.

**Step 1:**
Import Data

In [2]:
import json
import numpy as np
import pandas as pd

with open("Spirelogs") as f:
    data = json.load(f)

**Step 2:** Filter out extraneous data and separate the data into each individual game

In [4]:
watcher_data_raw = [x for x in data if x["ascension"] == 20 if x["character"] == "WATCHER"]

In [5]:
for x in watcher_data_raw:
    for y in range(len(x["cards"])):
        x["cards"][y] = x["cards"][y].replace("+1","")

In [6]:
diff = np.diff([x["floor"] for x in watcher_data_raw])
split_arr = np.where(diff<0)[0]+1
watcher_data = np.split(watcher_data_raw,split_arr)

**Step 3:** Calculate the health loss per combat associated with each deck

In [7]:
for run in watcher_data:
    damage = []
    total_enemies = []
    for floor in run:
        damage.append(floor["damage_taken"])
    for floor in run:
        floor["entering_hp"] = floor["entering_hp"]-floor["damage_taken"]
        floor["enemies"] = floor["enemies"].replace(" ","")
        total_enemies.append(floor["enemies"])
        avg = np.average(damage)
        damage.pop(0)
        floor["damage_taken"] = avg
        floor["enemies"] = np.copy(total_enemies)

**Step 4:** Encode Categorical Features

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer

cv_enemies = CountVectorizer()
mlb_relics = MultiLabelBinarizer()
cv_cards = CountVectorizer()

In [9]:
card_list,relic_list,enemy_list,floor_list,damage_list,hp_list  = [],[],[],[],[],[]

for run in watcher_data:
    for floor in run:
        if(floor["entering_hp"]!=0):
            card_list.append(" ".join((floor["cards"])))
            relic_list.append((floor["relics"]))    
            enemy_list.append(" ".join((floor["enemies"])))
            floor_list.append(floor["floor"])
            damage_list.append(floor["damage_taken"])
            hp_list.append(floor["entering_hp"])

In [11]:
card_values = cv_cards.fit_transform(card_list).toarray()
relic_values = mlb_relics.fit_transform(relic_list)
enemy_values = cv_enemies.fit_transform(enemy_list).toarray()

In [12]:
X = []
for i in range(len(card_list)):
    features =  np.concatenate([card_values[i],relic_values[i],enemy_values[i],[floor_list[i]],[hp_list[i]]])
    X.append(features)
y = damage_list

**Step 5:** Train the Multilayer Perceptron Model

In [13]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

**Step 6:** Hyperparameter Tuning <br /> <br />
Note: these cells will take a while to run, so running these cells can be bypassed by accessing the pickled data at **Step 8**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
param_1 = {
    'activation': ['relu', 'tanh', 'logistic'],
    'alpha': [1,0.1,0.01,0.001,0.0001]

}
param_2 = {
    'learning_rate_init': [0.001,0.01,0.1,0.2,0.3],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}


In [ ]:
mlp = MLPRegressor(random_state=1, max_iter=10**6)
search = RandomizedSearchCV(mlp,param_1,scoring='neg_mean_squared_error',cv=5)
search_result=search.fit(X_train,y_train)
search_result.best_params_

In [ ]:
mlp2 = MLPRegressor(random_state=1,activation = "relu", alpha = 0.1, max_iter=10**6)
search2 = RandomizedSearchCV(mlp2,param_2,scoring='neg_mean_squared_error',cv=5)
search2_result=search2.fit(X_train,y_train)
search2_result.best_params_

**Step 7:** Measure Performance: Using K-fold Validation, and Test Set Predictions

In [ ]:
sizes = [100,200,(100,100),(200,200),(200,200,200),(100,200,100),(200,100,200),(100,200,200)]
scores = []
scores_test =[]

In [ ]:
for size in sizes:
    mlp3 = MLPRegressor(random_state=1, hidden_layer_sizes = size, activation = "relu", alpha = 0.1,learning_rate_init= 0.01, learning_rate ="adaptive",max_iter=10**6)
    sc = cross_val_score(mlp3, X_train, y_train, scoring = "neg_mean_squared_error",cv=5)
    scores.append(np.average(sc))
    
    model = mlp3.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = np.average((predictions - y_test)**2)
    scores_test.append(mse)

In [ ]:
reg= [10,1,0.1,0.01,0.001,0.0001,0.00001]
reg_scores = []
reg_scores_test =[]

In [ ]:
for val in reg:
    mlp4 = MLPRegressor(random_state=1,  alpha= val, activation = "relu", hidden_layer_sizes = (100,200,200),learning_rate_init= 0.01, learning_rate ="adaptive",max_iter=10**6)
    sc = cross_val_score(mlp4, X_train, y_train, scoring = "neg_mean_squared_error",cv=5)
    reg_scores.append(np.average(sc))
    
    model = mlp4.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = np.average((predictions - y_test)**2)
    reg_scores_test.append(mse)

In [ ]:
import pickle

pickle.dump([search_result,search2_result,scores,reg_scores,reg,sizes,scores_test,reg_scores_test],open('save.p', 'wb'))

**Step 8:** Organize the Results

In [ ]:
import pickle
trial1,trial2,trial3,trial4,reg,sizes,scores_test,reg_scores_test = pickle.load(open( "save.p", "rb" ))

In [ ]:
trial1.best_params_

In [ ]:
trial2.best_params_

In [ ]:
sizes[np.argmax(trial3)]

In [ ]:
layers_data = {"Hidden Layer Sizes":sizes,"Mean Squared Error":-np.array(scores)}
df1 = pd.DataFrame(layers_data)

In [ ]:
regularization_data = {"Regularization Value":reg,"Mean Squared Error":-np.array(reg_scores)}
df2 = pd.DataFrame(layers_data)

In [ ]:
layers_test = {"Hidden Layer Sizes":sizes,"Mean Squared Error":scores_test}
df3 = pd.DataFrame(layers_test)

In [ ]:
regularization_test = {"Regularization Value":reg,"Mean Squared Error":reg_scores_test}
df4 = pd.DataFrame(regularization_test)

**Step 9:** Evaluate the Final Model

In [ ]:
mlp_final = MLPRegressor(random_state=1,  alpha= 0.1, activation = "relu", hidden_layer_sizes = (100,200,200),learning_rate_init= 0.01, learning_rate ="adaptive",max_iter=10**6)
final_fold = cross_val_score(mlp_final, X_train, y_train, scoring = "neg_mean_squared_error",cv=5)

In [ ]:
fold_score = np.average(-final_fold)
fold_score

In [ ]:
model = mlp_final.fit(X_train, y_train)
predictions = model.predict(X_test)
mse = np.average((predictions - y_test)**2)
mse

In [ ]:
regularization_test = {"L2 Regularization":[0.1],"Activation":["Relu"],"Hidden Layers":[(100,200,200)],"Learning Rate":["adaptive"],"Init. Rate":[.01],"K-fold (MSE)":[fold_score],"Test Set Error (MSE)":[mse]}
df4 = pd.DataFrame(regularization_test)
df4

**Results and Analysis**

Slay the Spire is a highly complex card game with an extremely large number of combinations of cards,relics, and enemies. Furthermore, no card operates in a vacuum, rather they each interact with different relics and enemies in unique ways. Therefore, one of my primary concerns when designing a machine learning algorithm was that my algorithm would be unable to learn these underlying patterns in the data and would instead “memorize” the data in the training set, effectively overfitting to this set. Therefore, when optimizing the hyperparameters in my model, I emphasized testing different values for L2 regularization and hidden layers sizes. By using L2 regularization I could impose a greater penalty for higher weight coefficients, thus reducing overfitting. Furthermore, I wanted to be able to reduce the number of hidden nodes in the model, since  creating an overly complex model with too many nodes would generalize poorly to unseen data.


Estimating the Performace of the Models on Unseen Data Using 5-Fold Cross Validation

In [ ]:
df1

In [ ]:
df2

The tables above demonstrate the performance of the regression models based on different regularization values and numbers of hidden nodes. The models are scored by using 5-fold validation to estimate their performance on unseen data. These trials indicate the best configuration for the hidden layer is (100,200,200), meaning 100 nodes in the 1st layer, 200 nodes in the 2nd and 3rd layers. This configuration is complex, with both a significant amount of nodes and layers. However, the 1st layer is small when compared to the size of the feature vector (267). This indicates information/complexity is lost when moving from the input layer to the first layer, which could contribute to the lower level of overfitting. The best choice among the given regularization values is 0.1. This implies that regularization values of 10 and 1 may impose too steep a penalty on the training model, resulting in an overly simplified model.

Evaluating Model Performance on the Test Set:

In [ ]:
df3

In [ ]:
df4

The final trial (shown above), tests the estimates made using the 5-fold validation by testing the models (using different values of L2 and hidden layers), on the actual test set. These results differ from the expectations, of all the hidden layer configurations, (100,200,200) performed second worst, with an MSE of 19.40. Additionally, the L2 value of 0.1 performed worst among these tests, whereas .001 performed the best, with an MSE of 16.41.

**Opportunities for Future Investigation** 

The discrepancies in performance in the two trials, K-fold validation and test, makes me believe that I can improve the model and reduce its variability by adding more data points to the set. The largest data set available when I started the project contained 1216 usable data points. However, an update released last month has made data for over 77 million new games easily accessible. This would provide an opportunity to explore how new data would affect my model. 


**Implications**

Machine learning algorithms have gained notoriety for their ability to learn traditional strategy games such as Chess or Go and beat highly-skilled human opponents. However, AI served not only to compete against opponents, but it also provided a tool for players to better understand the game. For the card/strategy video game industry, machine learning could lead to similar advancements. For newer players, it could help identify the underlying strategies and concepts of the game. Even for experienced players, ML algorithms can reveal innovative and unconventional ways of playing games.

**References** <br/>
https://spirelogs.com/ - For gathering raw data


https://towardsdatascience.com/bringing-deep-neural-networks-to-slay-the-spire-a2971d5a5115 
This site helped me figure out the strategy for converting card/relics/enemies information into feature values. I apted this method, but using different encoding methods, different choices for features, and I computed different target values. 